In [ ]:
print("this is a test" + chr(0) + "string")

In [ ]:
test_string = "hello! こんにちは!"
utf8_encoded = test_string.encode("utf-8")
print(utf8_encoded)
print(type(utf8_encoded))
list(utf8_encoded)

print(len(test_string))
print(len(utf8_encoded))

print(utf8_encoded.decode("utf-8"))

In [ ]:
import regex as re


PAT = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""

re.findall(PAT, "some text that i'll pre-tokenize")

In [ ]:
re.escape("<|endoftext|>")

In [ ]:
for match in re.finditer(PAT, "some text that i'll pre-tokenize"):
    print(f'"{match.group()}"')


In [ ]:
from cs336_basics.bpe.utils import PAT
import regex as re

text = """
```py
from cs336_basics.bpe.utils import PAT
import regex as re

for match in re.finditer(PAT, "some text that i'll pre-tokenize"):
    print(f'"{match.group()}"')
```

"""

for match in re.finditer(PAT, text):
    print(f'"{match.group()}"')


In [ ]:
from cs336_basics.bpe.train import get_pretoken_counts

counts = get_pretoken_counts("data/TinyStoriesV2-GPT4-train.txt", ["<|endoftext|>"])
len(counts)

In [ ]:
sorted(counts.items(), key=lambda x: x[1], reverse=True)

In [ ]:
"""

(1, 2, 3, 4): 5,
(2, 4, 4, 5): 4,
(1, 2, 3, 5): 3,
(1, 2, 4, 5): 2,
(1, 3, 4, 5): 1,    

1 2 -> [ (1, 2, 3, 4), (1, 2, 3, 5), (1, 2, 4, 5) ]

---

(12, 3, 4): 5,
(2, 4, 4, 5): 4,
(12, 3, 5): 3,
(12, 4, 5): 2,
(1, 3, 4, 5): 1,

1 2 -> [ (12, 3, 4), (12, 3, 5), (2, 4, 5) ]


2 3 -> [ (1, 2, 3, 4), (1, 2, 3, 5) ]


@dataclass
class TokenRef:
    tokens: list[bytes]
    count: int


class BytePair(NamedTuple):
    left: bytes
    right: bytes


# Got from pretokenization
list token_refs (list[TokenRef]) token_refs

# Compute below 2 data structures from token_refs
- dict byte_pair BytePair -> total_count (int) bp_to_counts
- dict byte_pair BytePair -> token_ref_ids (set[int]) bp_to_token_ref_ids

while less than vocab_size:
    bp = BytePair with highest count
    add it to the vocabulary

    token_refs_with_bp = find list of token refs contain this bp (use bp_to_token_ref_ids and token_refs)

    for each token_ref in token_refs_with_bp:
        compute curr_bp_to_counts for this token_ref
        perform merging on pretokenized text chunk
        compute next_bp_to_counts for this token_ref

        for each bp in curr_bp_to_counts:
            next_count = 
            if bp not in next_bp_to_counts:
                remove token_ref id from bp_to_token_ref_ids[bp]

            curr_count = curr_bp_to_counts[bp]
            next_count = 0 if bp not in next_bp_to_counts else next_bp_to_counts[bp]
            bp_to_counts[bp] += next_count - curr_count
            !assert bp_to_counts[bp] never be negative
        
    bp_count, bp_ref_count = bp_to_counts[bp], len(bp_to_token_ref_ids[bp])
    !assert bp_count == bp_ref_count == 0

    remove bp from bp_to_counts
    remove bp from bp_to_token_ref_ids

"""